# **Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones**

## **Edición 2023**


## Análisis exploratorio y curación de datos

### Trabajo práctico entregable - Grupo 22 - Parte 2

**Integrantes:**
- Chevallier-Boutell, Ignacio José
- Ribetto, Federico Daniel
- Rosa, Santiago
- Spano, Marcelo

**Seguimiento:** Meinardi, Vanesa

---

## Librerías

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.width', 1000)
pd.options.mode.chained_assignment = None  # default='warn'

import seaborn as sns
sns.set_context('talk')
sns.set_theme(style='white')

from sklearn.preprocessing import OneHotEncoder
import missingno as msno

from sklearn.experimental import enable_iterative_imputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import IterativeImputer

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

scaler = MinMaxScaler()
robust = RobustScaler()

def normalizer(data):
    Media = data.mean()
    SD = data.std()
    normData = (data - Media) / SD

    return normData, Media, SD

def denormalizer(data, m, s):
    return data * s + m

## Acerca del dataset

En la parte 1 del entregable se seleccionaron aquellas filas y columnas que consideramos relevantes para el problema de predicción de los precios de las propiedades en Melbourn, Australia. Utilizaremos dicho conjunto de datos resultante.

In [ ]:
df = pd.read_csv('GuardadoFinal.csv').iloc[:, 1:]
df[:3]

---
# Ejercicio 1 - Encoding

En la mayoría de los modelos de machine learning es necesario que las variables que se utilizan para entrenarlo sean del tipo numéricas. Por este motivo, suele ser necesario encontrar algún mapeo útil que permita transformar a la variables categóricas en numéricas.

En este caso las variables categóricas que consideramos importantes para la predicción del precio de las casas son CouncilArea, Regionname, SellerG y Type. Las 4 son variables nominales ya que no tienen un orden en sus categorías. En este sentido, consideramos que el algoritmo one-hot encoding es útil para realizar su codificación. El mismo crea una ristra de números con tantas cifras como categorías posea la variable considerada: cuando el registro pertenece a una dada categoría, se genera un 1 en dicha posición, siendo el resto de las cifras iguales a cero.

Vamos a comenzar el proceso de codificación separando entre variables categóricas y numéricas, según lo antes mencionado. Luego, vemos la cantidad de categorías que posee cada una de las variables categóricas elegidas y el número de columnas que se creará en total luego de realizar la codificación one-hot: mapearemos las 4 columnas categóricas en 41 columnas numéricas.

In [ ]:
categorical_cols = ['CouncilArea', 'Regionname', 'SellerG', 'Type']
numerical_cols = [x for x in df.columns if (x not in categorical_cols) and x not in ['Postcode', 'zipcode']]

print('Cantidad de categorías para cada variable:')
print(df[categorical_cols].nunique())
print('')
print('Cantidad de columnas que se creará con One Hot Encoding:', df[categorical_cols].nunique().sum())

Antes de pasar a la codificación, corroboramos la presencia de datos faltantes utilizando la librería `missingno`. En el gráfico de barras vemos que en CouncilArea faltan 1355 datos, representando el 10% del total de registros. Estos registros recibirán una categória propia dentro de esta variable cuando hagamos la codificación one-hot.

In [ ]:
fig, axs = plt.subplots(figsize=(6, 5))
msno.bar(df[categorical_cols], sort="ascending", fontsize=12, color="tab:green", ax=axs)
axs.set_ylim(0.8, 1)
plt.show()

A continuación utilizamos la función OneHotEncoder de sklearn para realizar el One Hot Encoding de las variables. En el código se describe el paso a paso, pero la idea final es crear un nuevo DataFrame de Pandas con las nuevas columnas antes dichas.

In [ ]:
# Nos quedamos con la columnas categóricas del DataFrame
features = df[categorical_cols]
# Creamos una lista con las categorías de cada variable categórica
categories = [features[column].unique() for column in features.columns]
# Inicializamos el enconder
encoder = OneHotEncoder(categories=categories)
# Mapeamos las categóricas a one-hot
encoded_features = encoder.fit_transform(features)

# Creación de nuevas columnas para one-hot
feature_names = []
for i, column in enumerate(features.columns):
    for category in categories[i]:
        feature_names.append(f'{column}_{category}')

encoded_df = pd.DataFrame(encoded_features.toarray(), columns=feature_names)
encoded_df.sample(10).T

Para finalizar este punto, unimos las variables numéricas originales con las categóricas codificadas.

In [ ]:
new_df = pd.concat([encoded_df, df[numerical_cols]], axis=1)

new_df.sample(5).T

---
# Ejercicio 2 - Imputación por KNN

## Análisis preliminar con `missingno`

En este ejercicio trabajaremos sobre las variables numéricas, imputando de alguna manera en aquellos registros donde tengamos valores faltantes. Para empezar, creamos un DataFrame conteniendo las columnas de interés (todas menos aquellas que tienen información de AirBnB) y analizamos con `missingno`.

A partir del gráfico de barras vemos que YearBuilt y BuildingArea presentan datos faltantes 60(faltan el 40% y el 48%, respectivamnete). Luego, como el nuevo DataFrame está ordenado en función de BuildingArea, el gráfico de matriz nos muestra que hay una gran correlación entre los datos faltantes en estas 2 categorías bajo análisis: la gran mayoría de datos faltantes en YearBuilt se corresponden con datos faltantes en BuildingArea. Esta idea queda clara cuando pasamos al mapa de calor, el cual mide la correlación de nulidad: qué tan fuerte la presencia (o ausencia) de una variable afecta la presencia de otra. Las variables que están completamente llenas o completamente vacías no presentan correlación significativa, así que quedan automáticamente descartadas de la gráfica. Además, la gráfica sólo completa las correlaciones en la triangular inferior. La gráfica nos da un valor de 0.8, lo cual se interpreta de la siguiente manera: es altamente probable que cada vez que un registro tiene un valor no nulo en YearBuilt también tenga un valor no nulo BuildingArea.

In [ ]:
#separamos el df entre lo que tiene información de AirBnB y lo que no. Va a ser útil después para unir todas las tablas en el problema 4.

#primero hago el sort por BuildingArea
new_df = new_df.sort_values('BuildingArea')

numcol_airless = [x for x in numerical_cols if (x.split('_')[0] != 'airbnb')]
df_airless = new_df[numcol_airless]#.sort_values('BuildingArea')

numcol_air = [x for x in numerical_cols if (x.split('_')[0] == 'airbnb')]
df_air = new_df[numcol_air]


msno.bar(df_airless, sort="ascending", fontsize=12, color="tab:green", figsize=(6, 5))
msno.matrix(df_airless, fontsize=12, color=[0.5,0,0], figsize=(6, 5))
msno.heatmap(df_airless, fontsize=12, figsize=(6, 5))

plt.show()

## Primeras pruebas con el imputador

Ahora que sabemos dónde hay datos faltantes y sus posibles conexiones, vamos a pasar al tratamiento de los mismos, inclinándonos por el camino de la imputación. La técnica de imputación a utilizar se clasifica como avanzada, ya que reemplazaremos los datos faltantes por algún valor sustituto, estimado por un algortimso de  aprendizaje automático.

Particularmente utilizaremos la imputación iterativa de sklearn (`IterativeImputer`), basada en la imputación multiple por ecuaciones encadenadas (MICE): el imputador modela cada variable con valores faltantes como una función de las otras variables, a partir de las cuales estima la imputación. La iteración la hace de manera rotatoria, generando un todos-vs-todos: en cada paso una columna es designada como output *y*, mientras que las otras columnas son tratadas como input $X$, sobre las cuales se ajusta un regresor (`estimator`) sobre las *y* conocidas para poder después predecir los valores faltantes en *y*. Esta predicción se realiza mediante regresiones múltiples sobre una muestra aleatoria de los datos y, luego, toma el promedio de los valores de regresión múltiple y usa ese valor para imputar el valor faltante. Este tipo de imputación funciona llenando los datos faltantes varias veces: todo el proceso se repite para cada variable durante `max_iter` rondas, siendo las salidas de la última ronda de iteración los resultados finales del proceso.

El regresor a utilizar será `KNeighborsRegressor`, el cual se basa en el algoritmo de k vecinos más próximos (KNN): se basa en la *similitud de características* para predecir los valores de cualquier nuevo punto de datos. Esto significa que al nuevo punto se le asigna un valor en función de su parecido con los puntos del conjunto de entrenamiento, siendo muy útil para hacer predicciones sobre valores faltantes al encontrar los k-vecinos más cercanos a la observación con datos perdidos y luego imputarlos en función de los valores no perdidos en el *vecindario*. Observamos que utilizar `IterativeImputer` con el regresor `KNeighborsRegressor` **no** es equivalente a utilizar el imputador `KNNImputer` (también de sklearn): aunque ambos están basados en KNN, uno es un imputador en sí mismo y otro es un regresor utilizado por otro imputador. En otros términos, `KNeighborsRegressor` es el método de predicción de valores faltantes, mientras que `KNNImputer` es el método de reemplazo de valores faltantes.

En ambas estrategias de imputación hay que realizar encoding para terminar teniendo todas variables numéricas. Además, es necesario realizar una estandarización, ya que datos con diferentes escalas introducen valores de reemplazo sesgados. Puntualmente, el imputador iterativo asume una distribución Gaussiana sobre las variables de salida, por lo que las características deberían ser normales o ser transformadas para hacerlas lo más normales posibles, mejorando el desempeño del imputador. Si bien el `KNNImputer` es rápido y fácil, no es muy preciso ni tiene cómputo para el error. Por su parte, `IterativeImputer` es mucho más versátil, ya que puede utilizarse con diferentes tipos de regresores. Asimismo, las imputaciones múltiples son mucho mejores que una única imputación (como con `KNNImputer`), ya que mide la incertidumbre de los valores perdidos de una mejor manera.

Una útima consideración antes de instanciar el imputador es que el `IterativeImputer` es sensible a la tolerancia, la cual a su vez está relacionada al regresor a utilizar. El valor de tolerancia por defecto es de 1e-3, pero la documentación recomienda usar una tolerancia del orden de 1e-2 cuando el regresor es `KNeighborsRegressor`. Comenzamos usando `max_iter=10` con `tol=5e-2` y resultó convergente para el caso sin escalear. Sin embargo, para poder converger cuando se tenía en cuenta el reescaleo tuvimos que relajar las tolerancia y extender las iteraciones, sino caíamos siempre en un `early stopping`. La única manera en que todos los casos fueran simultáneamente convergentes bajo el mismo imputador fue usando `max_iter=50` con `tol=2e-1`. Consideramos que esta no es la mejor situación ya que habrá casos donde estamos llegando a una solución subóptima. Decidimos entonces generar imputadores _ad hoc_.

In [ ]:
# Instanciamos el IterativeImputer con un estimador `KNeighborsRegressor`
mice_imputer = IterativeImputer(estimator=KNeighborsRegressor(), max_iter=10, 
                                tol=5e-2, random_state=0)

Probamos el imputador con los datos *crudos*, *i.e.* sin ningún tipo de pretratamiento.

In [ ]:
mice_knn = df_airless.copy(deep=True)
mice_knn[['YearBuilt','BuildingArea']] = mice_imputer.fit_transform(mice_knn[['YearBuilt', 'BuildingArea']])

Al ver el gráfico de barras de `missingno`, tenemos que efectivamente los valores faltantes han sido imputados por algún otro valor. Los histogramas nos indican qué valores han tomado las imputaciones nuevas.

In [ ]:
msno.bar(mice_knn, sort="ascending", fontsize=12, color="tab:green", figsize=(6, 5))
plt.show()

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(12,5))

sns.histplot(mice_knn['YearBuilt'], ax=axs[0], binwidth=5, color='tab:orange', label='Imputado', kde=True)
sns.histplot(df_airless['YearBuilt'], ax=axs[0], binwidth=5, color='tab:green', label='Original', kde=True)
axs[0].set_ylabel("")
axs[0].legend()

sns.histplot(mice_knn['BuildingArea'], ax=axs[1], binwidth=20, color='tab:orange', label='Imputado', kde=True)
sns.histplot(df_airless['BuildingArea'], ax=axs[1], binwidth=20, color='tab:green', label='Original', kde=True)
axs[1].set_ylabel("")
axs[1].legend()

plt.show()

## Comparación de escaleos

Vamos a evaluar el desempeño del imputador frente a diferentes tipos de escaleo:
* Normalización (`normalizer`): restar la media y dividir por la desviación estándar.
* Escaleo simple (`MinMaxScaler`): escalea los valores al rango [0, 1].
* Escaleo robusto (`RobustScaler`): similar al anterior, pero usando estadística. Así es más robusto a *outliers*.

Generamos entonces las imputaciones en función de cada escaleo.

In [ ]:
mice_imputer = IterativeImputer(estimator=KNeighborsRegressor(), max_iter=40, 
                                tol=1e-1, random_state=0)


Media, SD = [], []
# Creamos un nuevo DataFrame
mice_knn_norm = df_airless.copy(deep=True)

# Normalizamos
mice_knn_norm['YearBuilt'], m, s = normalizer(mice_knn_norm['YearBuilt'])
Media.append(m)
SD.append(s)
mice_knn_norm['BuildingArea'], m, s = normalizer(mice_knn_norm['BuildingArea'])
Media.append(m)
SD.append(s)

# Imputamos
mice_knn_norm[['YearBuilt','BuildingArea']] = mice_imputer.fit_transform(mice_knn_norm[['YearBuilt', 'BuildingArea']])

# Revertimos la normalización
mice_knn_norm['YearBuilt'] = denormalizer(mice_knn_norm['YearBuilt'], Media[0], SD[0])
mice_knn_norm['BuildingArea'] = denormalizer(mice_knn_norm['BuildingArea'], Media[1], SD[1])


In [ ]:
mice_imputer = IterativeImputer(estimator=KNeighborsRegressor(), max_iter=50, 
                                tol=2e-1, random_state=0)


# Creamos un nuevo DataFrame
mice_knn_sca = df_airless.copy(deep=True)
# Escaleamos
mice_knn_sca[['YearBuilt','BuildingArea']] = scaler.fit_transform(mice_knn_sca[['YearBuilt', 'BuildingArea']])
# Imputamos
mice_knn_sca[['YearBuilt','BuildingArea']] = mice_imputer.fit_transform(mice_knn_sca[['YearBuilt', 'BuildingArea']])
# Revertimos la transformación
mice_knn_sca[['YearBuilt','BuildingArea']] = scaler.inverse_transform(mice_knn_sca[['YearBuilt', 'BuildingArea']])

In [ ]:
mice_imputer = IterativeImputer(estimator=KNeighborsRegressor(), max_iter=40, 
                                tol=1e-1, random_state=0)


# Creamos un nuevo DataFrame
mice_knn_rob = df_airless.copy(deep=True)
# Escaleamos
mice_knn_rob[['YearBuilt','BuildingArea']] = robust.fit_transform(mice_knn_rob[['YearBuilt', 'BuildingArea']])
# Imputamos
mice_knn_rob[['YearBuilt','BuildingArea']] = mice_imputer.fit_transform(mice_knn_rob[['YearBuilt', 'BuildingArea']])
# Revertimos la transformación
mice_knn_rob[['YearBuilt','BuildingArea']] = robust.inverse_transform(mice_knn_rob[['YearBuilt', 'BuildingArea']])

Ahora comparamos los resultados entre el conjunto de datos original (`Original`), los datos imputados en crudo (`Imputado`) y los datos imputados luego de escalear con el escaleo simple (`Escaleado`), el escaleo robusto (`Robusto`) y la normalización (`Norm`).

Observamos que, efectivamente, es necesario el escaleo previo, ya que la respuesta de la imputación cambia al hacerlo. Más aún: la respuesta cambia en función del escaleo que hayamos elegido. Particularmente:
* El imputado crudo y el escaleado simple arrojan resultados similares, siendo que el escaleado requiere 5 veces más iteraciones y una tolerancia 4 veces mayor.
* El escaleado robusto y el normalizado otorgan resultados similares, requieriendo la misma cantidad de iteraciones y tolerancia entre sí, siendo 4 veces más iteraciones respecto al crudo y una tolerancia 2 veces mayor.

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(12,5))

sns.histplot(mice_knn['YearBuilt'], ax=axs[0], label='Imputado', binwidth=5, kde=True)
sns.histplot(mice_knn_sca['YearBuilt'], ax=axs[0], label='Escaleado', binwidth=5, kde=True)
sns.histplot(mice_knn_rob['YearBuilt'], ax=axs[0], label='Robusto', binwidth=5, kde=True)
sns.histplot(mice_knn_norm['YearBuilt'], ax=axs[0], label='Norm', binwidth=5, kde=True)
sns.histplot(df_airless['YearBuilt'], ax=axs[0], label='Original', binwidth=5, kde=True)
axs[0].set_ylabel("")
axs[0].legend()

sns.histplot(mice_knn['BuildingArea'], ax=axs[1], label='Imputado', binwidth=20, kde=True)
sns.histplot(mice_knn_sca['BuildingArea'], ax=axs[1], label='Escaleado', binwidth=20, kde=True)
sns.histplot(mice_knn_rob['BuildingArea'], ax=axs[1], label='Robusto', binwidth=20, kde=True)
sns.histplot(mice_knn_norm['BuildingArea'], ax=axs[1], label='Norm', binwidth=20, kde=True)
sns.histplot(df_airless['BuildingArea'], ax=axs[1], label='Original', binwidth=20, kde=True)
axs[1].set_ylabel("")
axs[1].legend()

plt.show()

## Conclusión

A partir de todo lo dicho creemos que sí es necesario hacer un escaleo previo a la imputación. Entre los métodos probados, nos inclinamos por el escaleo robusto aprovechando su robustez frente a *outliers* y el hecho de que sea una función ya implementada y estudiada por sklearn.

Ahora  llenamos los datos faltantes de nuestro dataset con los datos imputados:

---
# Ejercicio 3 y 4 - Reducción de dimensionalidad y composición del resultado

En este ejercicio deseamos encontrar las componentes principales y reducir la dimensionalidad de la matriz obtenida en el ejercicio anterior mediante la aplicación de la clase `PCA` de scikit-learn.

Las componentes principales son *direcciones* en el espacio de nuestros datos $R^{n}$ (más específicamente son combinaciones lineales de los datos). Estos componentes o direcciones se calculan de forma tal que:
* Son ortogonales, es decir, no están correlacionadas.
* Están ordenados de acuerdo al nivel de varianza de los datos originales que representan.

Usando estos componentes, se puede construir una proyección lineal de nuestros datos a una nueva matriz donde cada columna ahora está en la dirección de un componente principal. Luego, se selecciona un subconjunto de las primeras $d$ columnas y, por las propiedades de los componentes principales, sabemos que hemos perdido la menor cantidad de varianza de nuestros datos.

Las componentes principales de la matriz original son computadas mediante el método `fit`, a las que luego podemos acceder a través de los atributos de la instancia `pca`. Es recomendable también estandarizar o al menos escalar la matriz original para asegurar de que todas las variables estén en las mismas unidades y ninguna tenga un peso demasiado grande.

Vamos a comenzar aplicando `RobustScaler` a toda la matriz y haciendo un pairplot junto con el cálculo del coeficiente de correlación de Pearson para todas las características.

In [ ]:
# Creamos un nuevo DataFrame para aplicar "RobustScaler"
mice_knn_rob_total = mice_knn_rob.copy(deep=True)
print('dataframe shape:')
print(mice_knn_rob_total.shape)
mice_knn_rob_total.head()

In [ ]:
# Escaleamos el df completo
mice_knn_rob_total = robust.fit_transform(mice_knn_rob_total)
mice_knn_rob_total

In [ ]:
# Hacemos el pairplot junto con el cálculo de el coef. de Pearson
from scipy.stats import pearsonr
df_test = pd.DataFrame(mice_knn_rob_total,columns = ['Distance','Lattitude','Longtitude','YearBuilt','Landsize','BuildingArea','Rooms','Bedroom2','Bathroom','Price'])
def corrfunc(x, y, ax=None, **kws):
    """Plot the correlation coefficient in the top left hand corner of a plot."""
    r, _ = pearsonr(x, y)
    ax = ax or plt.gca()
    ax.annotate(f'ρ = {r:.2f}', xy=(.1, .9), xycoords=ax.transAxes)

g = sns.pairplot(data=df_test)
g.map_lower(corrfunc)
plt.show()

Vemos que `Rooms` y `Bedroom2` son las características más correlacionadas: $\rho = 0.95$.

Ahora aplicamos PCA sin buscar reducir la dimensión de la matriz original (esto es, no hay pérdida de información).


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=None)
pca.fit(mice_knn_rob_total)
print("Componentes principales:")
print(pca.components_)

# Llevamos la matriz al espacio de componentes principales
mice_knn_rob_total_pca = pca.transform(mice_knn_rob_total)

print("Dimensión de la matriz transformada:")
print(mice_knn_rob_total_pca.shape)

Las componentes principales están ordenadas de mayor a menor en el sentido de información que aportan. La fracción de varianza explicada es una cantidad que mide la proporción de varianza en los datos que es explicada por cada componente principal. En otras palabras, esta cantidad nos permite ver cuánto aporta cada componente. También es interesante analizar cuánto es el acumulado de todas las componentes para poder establecer un criterio como, por ejemplo, quedarse con el número mínimo de componentes que aseguren un 90% de la información original.

In [ ]:
print("Varianza explicada:")
print(pca.explained_variance_)
print()
print("Razón de varianza explicada:")
evr = pca.explained_variance_ratio_
print(evr)
print()
print("porcentaje de la variable explicada:")
print(np.sum(evr))

Vamos a graficar la fracción de varianza que aporta cada componente y la información acumulada.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (12, 4))

ax[0].plot(range(1, len(evr) + 1), evr, '.-', markersize = 10)
ax[0].set_ylabel('Fracción de varianza explicada')
ax[0].set_xlabel('Número de componente principal')

varianza_acumulada = np.cumsum(evr)
ax[1].plot(range(1, len(evr) + 1), varianza_acumulada, '.-', markersize = 10)
ax[1].set_ylabel('Fracción acumulada de varianza explicada')
ax[1].set_xlabel('Cantidad de componentes principales')

fig.subplots_adjust(top=1.05)

En el segundo gráfico vemos que, reteniendo solamente 6 componentes principales, mantenemos aproximadamente el 90% de la información original.

Veamos qué nos dicen las primeras 2 componentes (las cuales contienen más del 50% de la información), con el fin de luego realizar un análisis gráfico en 2 dimensiones. Es decir, queremos usar PCA para visualizar los datos en un espacio de dimensión reducida.

In [ ]:
print('Features: ','Distance','Lattitude','Longtitude','YearBuilt','Landsize','BuildingArea','Rooms','Bedroom2','Bathroom','Price')
print('PCA1 = {}'.format(pca.components_[0]))
print('PCA2 = {}'.format(pca.components_[1]))

En la primera componente las características que más peso tienen son `Rooms` y `Bedroom2`, las cuales tienen un peso muy parecido. Esto tiene sentido ya que como vimos previamente en el pairplot, está variables tienen un coeficiente de correlación alto.

En la segunda componente la característica que más importa es `YearBuilt`.

Veamos ahora los datos en el espacio de estas dos primeras componentes principales:

In [ ]:
features = ['Distance','Lattitude','Longtitude','YearBuilt','Landsize','BuildingArea','Rooms','Bedroom2','Bathroom','Price']
features_pc = pca.components_.T

fig, ax = plt.subplots(figsize = (12, 12))

# Hacemos un scatter de los datos en las dos primeras componentes
ax.scatter(mice_knn_rob_total_pca[:,0], mice_knn_rob_total_pca[:,1], alpha = 0.65)

# Hacemos el grafico de las flechas indicando las direcciones de los features originales
sf = 5 # Factor de escala para agrandar las flechas

for i in range(len(features)):

  ax.arrow(0, 0, sf * features_pc[i][0], sf * features_pc[i][1], width = 0.1, color = 'g', alpha = 0.5)
  ax.text(sf * features_pc[i][0], sf * features_pc[i][1], s = features[i], fontdict= {'color': 'k', 'size': 10})

ax.set_xlabel('Primera componente principal')
ax.set_ylabel('Segunda componente principal')

Como era de esperar, vemos que `Rooms` y `Bedroom2` son casi paralelas a la primera componente principal, mientras que `YearBuilt` es casi paralela a la segunda componente principal.

Finalmente, agregaremos las 6 primeras componentes principales al conjunto de datos original. Para ello primeros convertimos el array transformado a un dataframe nuevo:

In [ ]:
names = ['PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10']
df_pca = pd.DataFrame(mice_knn_rob_total_pca,columns = names)
print(df_pca.shape)
df_pca.head()

Y ahora unimos las primeras 6 columnas al dataframe original, y sumamos los datos de la tabla de AirBnB para tener la composición final del resultado del ejercicio 4:

In [ ]:
df_join1 = mice_knn_rob.copy()
df_join2 = df_pca.copy() 

for name in ['PC1','PC2','PC3','PC4','PC5','PC6']:
    df_join1[name] = df_join2[name].values

df_join1

final_df = pd.concat([df_join1, df_air], axis=1)


print(final_df.keys())

---
# Ejercicio 5 - Documentación

En un documento `.pdf` o `.md` realizar un reporte de las operaciones que realizaron para obtener el conjunto de datos final. Se debe incluir:
  1. Criterios de exclusión (o inclusión) de filas
  2. Interpretación de las columnas presentes
  2. Todas las transofrmaciones realizadas

Este documento es de uso técnico exclusivamente, y su objetivo es permitir que otres desarrolladores puedan reproducir los mismos pasos y obtener el mismo resultado. Debe ser detallado pero consiso. Por ejemplo:

```
  ## Criterios de exclusión de ejemplos
  1. Se eliminan ejemplos donde el año de construcción es previo a 1900

  ## Características seleccionadas
  ### Características categóricas
  1. Type: tipo de propiedad. 3 valores posibles
  2. ...
  Todas las características categóricas fueron codificadas con un
  método OneHotEncoding utilizando como máximo sus 30 valores más 
  frecuentes.
  
  ### Características numéricas
  1. Rooms: Cantidad de habitaciones
  2. Distance: Distancia al centro de la ciudad.
  3. airbnb_mean_price: Se agrega el precio promedio diario de 
     publicaciones de la plataforma AirBnB en el mismo código 
     postal. [Link al repositorio con datos externos].

  ### Transformaciones:
  1. Todas las características numéricas fueron estandarizadas.
  2. La columna `Suburb` fue imputada utilizando el método ...
  3. Las columnas `YearBuilt` y ... fueron imputadas utilizando el 
     algoritmo ...
  4. ...

  ### Datos aumentados
  1. Se agregan las 5 primeras columnas obtenidas a través del
     método de PCA, aplicado sobre el conjunto de datos
     totalmente procesado.
```
